# 1. 키워드 추출

## 1-1 제목에서 추출

In [1]:
# 불필요한 경고 출력을 방지
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split


In [3]:
# '-' 깨짐 해결 모듈
import matplotlib.pylab as plt

import matplotlib as mpl

plt.rcParams["font.family"] = "Malgun Gothic"  # For Windows
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (8,4)
print(plt.rcParams["font.family"])

# 마이너스 깨짐 해결
mpl.rcParams["axes.unicode_minus"] = False

['Malgun Gothic']


### 데이터 읽기

In [4]:
# techworld
df_bloter = pd.read_csv('../data_crawling_D/before_keywords/news_cwn_data_202306231600.csv')
print(df_bloter.shape)

(1655, 8)


In [5]:
df_bloter.head()


,news_date,news_title,news_text_sm,url_in,news_writer,tags_string,thumb_addr,news_site
0,2023.05.30 17:10,웹3 메타버스 캐리버스 시네마틱 공개와 동시에 사전예약 돌입,㈜캐리버스가 웹3 메타버스 게임 ‘캐리버스’의 인게임 시네마틱 영상을 공개하며 글로...,https://www.cwn.kr/news/articleView.html?idxno...,김지영,"#캐리버스,#블록체인,#웹3 메타버스",https://www.cwn.kr/news/thumbnail/202305/16873...,CWN
1,2023.05.30 13:15,한국레노버 8세대 요가 신제품 3종 출시,레노버 요가 프로 9i한국레노버(대표 신규식)가 콘텐츠 크리에이터를 위한 뛰어난 성...,https://www.cwn.kr/news/articleView.html?idxno...,온라인뉴스팀,"#한국레노버,#노트북,#요가 프로 9i,#요가 프로 7i,#요가 슬림 7i 카본",https://www.cwn.kr/news/thumbnail/202305/mn_16...,CWN
2,2023.05.30 13:14,SK텔레콤 양자보안 폰 ‘갤럭시 퀀텀4’ 출시,SK텔레콤(대표이사 사장 유영상)은 삼성전자와 함께 양자보안 5G 스마트폰 ‘갤럭시...,https://www.cwn.kr/news/articleView.html?idxno...,온라인뉴스팀,"#SK텔레콤,#삼성전자,#양자보안 5G 스마트폰,#갤럭시 퀀텀 4",NaN,CWN
3,2023.05.30 12:07,Arm 차세대 GPU·CPU 공개…모바일 기기 성능 업그레이드 기대,출처: Arm로이터 테크크런치 더레지스터 등 복수 외신은 칩 제조사 Arm이 컴퓨텍...,https://www.cwn.kr/news/articleView.html?idxno...,최은희,"#Arm,#Arm Cortex-X4,#Arm Immortalis-G720,#TCS23",https://www.cwn.kr/news/thumbnail/202305/16863...,CWN
4,2023.05.30 10:12,아이폰16 프로 모델 2종 디스플레 크기 커질까?,출처: Apple맥루머스 GSM 아레나 등 복수 외신은 아이폰 16 프로 모델의 화...,https://www.cwn.kr/news/articleView.html?idxno...,고다솔,"#애플,#아이폰,#아이폰16 프로,#아이폰16 프로맥스",https://www.cwn.kr/news/thumbnail/202305/16861...,CWN


In [6]:
df_bloter.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1655 entries, 0 to 1654
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   news_date     1655 non-null   object
 1   news_title    1655 non-null   object
 2   news_text_sm  1654 non-null   object
 3   url_in        1655 non-null   object
 4   news_writer   1655 non-null   object
 5   tags_string   1501 non-null   object
 6   thumb_addr    1634 non-null   object
 7   news_site     1655 non-null   object
dtypes: object(8)
memory usage: 103.6+ KB


In [7]:
# df_techworld.columns
# idx 떼내기
df_bloter = df_bloter[[ 'news_date', 'news_title', 'news_text_sm', 'url_in',
       'news_writer', 'tags_string', 'thumb_addr', 'news_site']]

In [8]:
# df_recentit news_date 수정하기 (맨우측에 . 삭제)
df_bloter['news_date']= df_bloter['news_date'].str.replace(r'\.', '-', regex=True)
df_bloter['news_date']

# [ 출처: https://seong6496.tistory.com/ ]

0       2023-05-30 17:10
1       2023-05-30 13:15
2       2023-05-30 13:14
3       2023-05-30 12:07
4       2023-05-30 10:12
              ...       
1650    2022-05-31 16:29
1651    2022-05-31 16:16
1652    2022-05-31 14:14
1653    2022-05-31 14:01
1654    2022-05-31 12:22
Name: news_date, Length: 1655, dtype: object

In [9]:
# 결측값 채우기
df_bloter.fillna('', inplace=True)
# df_itworld.fillna('')

## KoNLPy로 단어 단위 분해

In [10]:
# datetime(날짜), os 호출

import datetime
import os
# 불용.txt 읽기 위해 csv 파일 호출
import csv
import re
# konlpy 호출
from konlpy.tag import Okt

# 자연어, 불용어 처리 및 wordcloud 생성을 위해
# 집계 메소드
from collections import Counter
from konlpy.tag import Komoran
# https://datascienceschool.net/03%20machine%20learning/03.01.01%20NLTK%20%EC%9E%90%EC%97%B0%EC%96%B4%20%EC%B2%98%EB%A6%AC%20%ED%8C%A8%ED%82%A4%EC%A7%80.html
from nltk.tokenize import word_tokenize


In [11]:
# 모듈화
def get_title_keywords(df):
    # 컬럼 'news_title' 가져오기 
    rdr = df['news_title'] 
    
    # Okt 객체 선언
    okt = Okt()
    ###  불용어 빼기
    # 불용어 파일 읽기
    stop_words = []
    with open('../data_analysis_test/불용.txt', 'r', encoding='utf-8') as file:
       stop_words = file.read().splitlines()
    
    h_word_list = []
    for w_bundle in rdr:
      # print(okt.nouns(w_bundle))
        list = okt.nouns(w_bundle)
        # 특수문자 제거
        tokens = ",".join(list)
        tokens = re.sub('[\{\}\[\]\/?.;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', tokens)
        list = tokens.split(",")
        # 한글자인 명사, ',' 빼기 (예외:웹,앱,폰,챗)
        for i,v in enumerate(list):
            if len(v) < 2 and (v != '웹' and v != '앱' and v != '폰' and v != '챗'):
                list.pop(i)
            elif v == ',':
                list.pop(i)
            # 불용어 제거
            elif i in stop_words:
                list.pop(i)
        list_str = ",".join(list)
       
        h_word_list.append(list_str)
    return h_word_list
    # print(h_word_list)
    # len(techworld_h_word_list)

bloter_h_word_list = get_title_keywords(df_bloter)

## 해당 리스트 목록을 새로운 컬럼으로 추가

In [12]:
df_bloter['news_key'] = bloter_h_word_list
df_bloter['news_key']

0             웹,메타,버스,캐리,버스,시네마,공개,동시,사전예약,돌입
1                         한국,레노버,세대,요가,신제품,출시
2                       텔레콤,양자,보안,폰,갤럭시,퀀텀,출시
3                   차세대,공개,모바일,기기,성능,업그레이드,기대
4                          아이폰,프로,모델,디스,플레,크기
                        ...                  
1650                      싱가포르,기업,코딩,실력,지원,선호
1651       맥도날드,드라이브,스루,챗봇,자연어,처리,기반,주문,기능,추가
1652                    델리,기반,교통,관리,시스템,배포,예고
1653    프라다,명품,브랜드,디지털,자산,발행,행렬,동참,타임캡슐,런칭,예고
1654            코딩,트렌드,다트,인기,프로그래밍,언어,랭킹,티,지수
Name: news_key, Length: 1655, dtype: object

## 1-2 news_text_sm에서 키워드 추출하기

In [13]:
# 모듈화
def get_text_keywords(df):
    # 컬럼 'news_title' 가져오기 
    rdr = df['news_text_sm'] 
    
    # Okt 객체 선언
    okt = Okt()
    ###  불용어 빼기
    # 불용어 파일 읽기
    stop_words = []
    with open('../data_analysis_test/불용.txt', 'r', encoding='utf-8') as file:
       stop_words = file.read().splitlines()
    
    h_word_list = []
    for w_bundle in rdr:
      # print(okt.nouns(w_bundle))
        list = okt.nouns(w_bundle)
        # 특수문자 제거
        tokens = ",".join(list)
        tokens = re.sub('[\{\}\[\]\/?.;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', tokens)
        list = tokens.split(",")
        # 한글자인 명사, ',' 빼기 (예외:웹,앱,폰,챗)
        for i,v in enumerate(list):
            if len(v) < 2 and (v != '웹' and v != '앱' and v != '폰' and v != '챗'):
                list.pop(i)
            elif v == ',':
                list.pop(i)
            # 불용어 제거
            elif i in stop_words:
                list.pop(i)
        list_str = ",".join(list)
       
        h_word_list.append(list_str)
    return h_word_list
    # print(h_word_list)
    # len(techworld_h_word_list)

bloter_txt_word_list = get_title_keywords(df_bloter)


In [14]:
df_bloter['news_txt_key'] = bloter_txt_word_list
df_bloter['news_txt_key']

0             웹,메타,버스,캐리,버스,시네마,공개,동시,사전예약,돌입
1                         한국,레노버,세대,요가,신제품,출시
2                       텔레콤,양자,보안,폰,갤럭시,퀀텀,출시
3                   차세대,공개,모바일,기기,성능,업그레이드,기대
4                          아이폰,프로,모델,디스,플레,크기
                        ...                  
1650                      싱가포르,기업,코딩,실력,지원,선호
1651       맥도날드,드라이브,스루,챗봇,자연어,처리,기반,주문,기능,추가
1652                    델리,기반,교통,관리,시스템,배포,예고
1653    프라다,명품,브랜드,디지털,자산,발행,행렬,동참,타임캡슐,런칭,예고
1654            코딩,트렌드,다트,인기,프로그래밍,언어,랭킹,티,지수
Name: news_txt_key, Length: 1655, dtype: object

In [15]:
# df_techworld

### 'news_key', 'news_txt_key' 값 합치기

In [16]:
df_bloter['total_keywords'] = df_bloter[['news_key', 'news_txt_key']].apply(','.join, axis=1)
print(df_bloter)

             news_date                                         news_title  \
0     2023-05-30 17:10                  웹3 메타버스 캐리버스 시네마틱 공개와 동시에 사전예약 돌입   
1     2023-05-30 13:15                             한국레노버 8세대 요가 신제품 3종 출시   
2     2023-05-30 13:14                          SK텔레콤 양자보안 폰 ‘갤럭시 퀀텀4’ 출시   
3     2023-05-30 12:07              Arm 차세대 GPU·CPU 공개…모바일 기기 성능 업그레이드 기대   
4     2023-05-30 10:12                        아이폰16 프로 모델 2종 디스플레 크기 커질까?   
...                ...                                                ...   
1650  2022-05-31 16:29                     싱가포르 기업 84% "코딩 실력 있는 지원자 선호해"   
1651  2022-05-31 16:16         IBM 맥도날드 AI 드라이브 스루 챗봇에 자연어 처리 기반 주문 기능 추가   
1652  2022-05-31 14:14                          印 델리 AI 기반 교통관리 시스템 배포 예고   
1653  2022-05-31 14:01         프라다 명품 브랜드의 디지털 자산 발행 행렬 동참…타임캡슐 NFT 런칭 예고   
1654  2022-05-31 12:22  [코딩 트렌드] 다트(Dart) 5월 PyPL 인기 프로그래밍 언어 랭킹 19위.....   

                                           news_text_sm  \
0     ㈜캐리버스가 웹3 

### 중복 키워드 처리

In [17]:
# from collections import OrderedDict

# list = '티맥스,티베로,인텔,코리아,비즈니스,연속성,혁신,전략,웨비,개최,티맥스,티베로,인텔,코리아,비즈니스,연속성,혁신,전략,웨비,개최'
# list_sep = list.split(",")
# list_sep

# a = " ".join(OrderedDict.fromkeys(list_sep))
# print(a)




In [18]:
from collections import OrderedDict

t_keyword_list = []

for keys in df_bloter['total_keywords']:
    # print([keys]) # 일단 전부 ,로 split()해줘야 단어끼리로 분리됨
    keys_list = keys.split(",")
    key_list_str = ','.join(OrderedDict.fromkeys(keys_list))
    # print(key_list_str)
    # 2차원 리스트 만들기
    t_keyword_list.append(key_list_str)

t_keyword_list


['웹,메타,버스,캐리,시네마,공개,동시,사전예약,돌입',
 '한국,레노버,세대,요가,신제품,출시',
 '텔레콤,양자,보안,폰,갤럭시,퀀텀,출시',
 '차세대,공개,모바일,기기,성능,업그레이드,기대',
 '아이폰,프로,모델,디스,플레,크기',
 '기술,혁신,연구소,연구,상업,활용,오픈,소스,팰컨,출시',
 '폭스바겐,소프트웨어,활용,자율,주행,서비스,탑승객,관리,시스템,개발',
 '해외,스타트업,노트북,개발,업무,혼합,현실,사용,대감',
 '마인,컴퍼니,기반,검색,솔루션,서핑,정식,출시',
 '에이수스,시리즈,그래픽카드,출시',
 '출시,임박,모토로라,더블,폰,화면,주사,지원,전망',
 '실제,마주,보고,클레,실시간,디지털,휴먼,챗봇,런칭',
 '의학,전문가,사용,위험성,경고',
 '애플,아이패드,공개,베타,버전,배포',
 '소니,다음,스마트폰,갤플,모습,클램셸,형태,더블,폰,출시,전망,제기',
 '메타,옥타곤,버스,버전,출시',
 '초슬림,경량,고성능,노트북,젠북,출시',
 '트루,테크놀로지,기업은행,실시간,신분증,사본,검증,솔루션,도입,추진,사업,수주',
 '에이수스,크리에이터,및,그래픽카드,출시',
 '라이,와이즈,챗,플러그인,출시,개인,맞춤,엔터테인먼트,추천,제공',
 '오픈,챗,아이폰,아이패드,버전,공식,앱,출시',
 '인트,플로우,돼지,분석,기술,특허,등록,완료',
 '퀸텟,시스,템즈,로우,코드,개발,플랫폼,대규모,업그레이드',
 '삼성,전자,업계,최선,나노,양산',
 '모토로라,플립,더블,레이저,폰,공개,예고',
 '안랩,사칭,기업,페이스북,계정,피싱,메일,주의,당부',
 '왓츠앱,사용자,프라이버시,채팅,기능,배포',
 '대화,챗봇,열풍,이면,열정,페이,논란',
 '에이,럭스,코딩,로봇,비누,신규,특허,추가,확보',
 '맥스,등,글로벌,기술,협력,글래스,적용,확대',
 '차세대,지능,로봇,연결,통합,테크,써밋,개최',
 '구글,가짜,이미지,확산,배포',
 '애플,프로,테스트,공정,채택,전망',
 '젤리,스페이스,원더풀,플랫폼,인공,지능,로봇,산업,

In [19]:
df_bloter['total_keywords'] = t_keyword_list
df_bloter['total_keywords'] 

0                웹,메타,버스,캐리,시네마,공개,동시,사전예약,돌입
1                         한국,레노버,세대,요가,신제품,출시
2                       텔레콤,양자,보안,폰,갤럭시,퀀텀,출시
3                   차세대,공개,모바일,기기,성능,업그레이드,기대
4                          아이폰,프로,모델,디스,플레,크기
                        ...                  
1650                      싱가포르,기업,코딩,실력,지원,선호
1651       맥도날드,드라이브,스루,챗봇,자연어,처리,기반,주문,기능,추가
1652                    델리,기반,교통,관리,시스템,배포,예고
1653    프라다,명품,브랜드,디지털,자산,발행,행렬,동참,타임캡슐,런칭,예고
1654            코딩,트렌드,다트,인기,프로그래밍,언어,랭킹,티,지수
Name: total_keywords, Length: 1655, dtype: object

### 'news_key', 'news_text_key' 컬럼 삭제

In [20]:
df_bloter.drop(['news_key', 'news_txt_key'], axis=1, inplace=True)

In [21]:
# df_techworld

## df_cwn csv 파일에 반영


In [22]:
df_bloter.to_csv("../data_crawling_D/after_keywords/news_cwn_k_t.csv", mode='w')

# 2. 신뢰도 평가 : news_titlw, news_text_sm의 데이터를 KRWORDRANK로 추출

In [23]:
import pandas as pd
import numpy as np

In [24]:
# 먼저 news_text_sm 컬럼의 값들을 뭉쳐서 하나의 string(각 요소는 '\n'로 나누기)
# 으로 통합
import csv # sql_cwn_data에서 갖고 오기

f = open("../data_crawling_D/after_keywords/news_cwn_k_t.csv", "r", encoding='utf8')
reader = csv.reader(f)
next(reader)
# print(reader)
data = []
for row in reader:
    data.append(row)
    # print(row)
f.close()



In [25]:
# data


In [26]:
# dataframe으로 만들기
df_bloter = pd.DataFrame(data=data, columns=['idx','news_date', 'news_title', 'news_text_sm', 'url_in', 'news_writer', 'tags_string', 'thumb_addr', 'news_site', 'total_keywords'])  
# idx (인덱스) 제거
df_bloter = df_bloter[['news_date', 'news_title', 'news_text_sm', 'url_in', 'news_writer', 'tags_string', 'thumb_addr', 'news_site','total_keywords']]
df_bloter

,news_date,news_title,news_text_sm,url_in,news_writer,tags_string,thumb_addr,news_site,total_keywords
0,2023-05-30 17:10,웹3 메타버스 캐리버스 시네마틱 공개와 동시에 사전예약 돌입,㈜캐리버스가 웹3 메타버스 게임 ‘캐리버스’의 인게임 시네마틱 영상을 공개하며 글로...,https://www.cwn.kr/news/articleView.html?idxno...,김지영,"#캐리버스,#블록체인,#웹3 메타버스",https://www.cwn.kr/news/thumbnail/202305/16873...,CWN,"웹,메타,버스,캐리,시네마,공개,동시,사전예약,돌입"
1,2023-05-30 13:15,한국레노버 8세대 요가 신제품 3종 출시,레노버 요가 프로 9i한국레노버(대표 신규식)가 콘텐츠 크리에이터를 위한 뛰어난 성...,https://www.cwn.kr/news/articleView.html?idxno...,온라인뉴스팀,"#한국레노버,#노트북,#요가 프로 9i,#요가 프로 7i,#요가 슬림 7i 카본",https://www.cwn.kr/news/thumbnail/202305/mn_16...,CWN,"한국,레노버,세대,요가,신제품,출시"
2,2023-05-30 13:14,SK텔레콤 양자보안 폰 ‘갤럭시 퀀텀4’ 출시,SK텔레콤(대표이사 사장 유영상)은 삼성전자와 함께 양자보안 5G 스마트폰 ‘갤럭시...,https://www.cwn.kr/news/articleView.html?idxno...,온라인뉴스팀,"#SK텔레콤,#삼성전자,#양자보안 5G 스마트폰,#갤럭시 퀀텀 4",,CWN,"텔레콤,양자,보안,폰,갤럭시,퀀텀,출시"
3,2023-05-30 12:07,Arm 차세대 GPU·CPU 공개…모바일 기기 성능 업그레이드 기대,출처: Arm로이터 테크크런치 더레지스터 등 복수 외신은 칩 제조사 Arm이 컴퓨텍...,https://www.cwn.kr/news/articleView.html?idxno...,최은희,"#Arm,#Arm Cortex-X4,#Arm Immortalis-G720,#TCS23",https://www.cwn.kr/news/thumbnail/202305/16863...,CWN,"차세대,공개,모바일,기기,성능,업그레이드,기대"
4,2023-05-30 10:12,아이폰16 프로 모델 2종 디스플레 크기 커질까?,출처: Apple맥루머스 GSM 아레나 등 복수 외신은 아이폰 16 프로 모델의 화...,https://www.cwn.kr/news/articleView.html?idxno...,고다솔,"#애플,#아이폰,#아이폰16 프로,#아이폰16 프로맥스",https://www.cwn.kr/news/thumbnail/202305/16861...,CWN,"아이폰,프로,모델,디스,플레,크기"
...,...,...,...,...,...,...,...,...,...
1650,2022-05-31 16:29,"싱가포르 기업 84% ""코딩 실력 있는 지원자 선호해""",마카오 비즈니스닷컴이 싱가포르 평생 교육 기관 NTUC 러닝허브(NTUC Learn...,https://www.cwn.kr/news/articleView.html?idxno...,임온유,"#코딩 실력,#테크 분야 취업",https://www.cwn.kr/news/thumbnail/202205/10260...,CWN,"싱가포르,기업,코딩,실력,지원,선호"
1651,2022-05-31 16:16,IBM 맥도날드 AI 드라이브 스루 챗봇에 자연어 처리 기반 주문 기능 추가,영국 온라인 매체 더레지스터에 따르면 IBM이 맥도날드 인공지능(AI) 드라이브 스...,https://www.cwn.kr/news/articleView.html?idxno...,고다솔,"#IBM,#맥도날드,#AI 챗봇,#드라이브 스루,#자연어 처리",https://www.cwn.kr/news/thumbnail/202205/10268...,CWN,"맥도날드,드라이브,스루,챗봇,자연어,처리,기반,주문,기능,추가"
1652,2022-05-31 14:14,印 델리 AI 기반 교통관리 시스템 배포 예고,인도 테크 매체 애널리틱스 인디아 매거진에 따르면 델리 교통국(Delhi Traff...,https://www.cwn.kr/news/articleView.html?idxno...,임온유,"#델리 교통국,#ITMS,#AI,#인공지능,#교통 신호",https://www.cwn.kr/news/thumbnail/202205/10256...,CWN,"델리,기반,교통,관리,시스템,배포,예고"
1653,2022-05-31 14:01,프라다 명품 브랜드의 디지털 자산 발행 행렬 동참…타임캡슐 NFT 런칭 예고,출처: Prada구찌 발망 발렌시아가 등에 이어 프라다도 명품 브랜드의 NFT 발행...,https://www.cwn.kr/news/articleView.html?idxno...,이선영,"#프라다,#NFT,#이더리움,#타임캡슐,#카시우스 허스트,#오로라 블록체인 컨소시엄",https://www.cwn.kr/news/thumbnail/202205/10257...,CWN,"프라다,명품,브랜드,디지털,자산,발행,행렬,동참,타임캡슐,런칭,예고"


## news_title과 news_text_sm를 합친 컬럼 생성하기

In [27]:
df_bloter['title_n_text'] = df_bloter[['news_title', 'news_text_sm']].apply(','.join, axis=1)
# df_techworld['title_n_text']


In [28]:
# 제목과 내용을 합친 str 컬럼을 news_date와 같이 엮어 df화
df_bloter_dnt = df_bloter[['news_date', 'title_n_text']]
# csv 파일로 저장하기
df_bloter_dnt.to_csv("../data_analysis_test/test_data/cwn_dnt.txt", mode='w', header=False, index=False, sep='\t')

In [29]:
# df_cwn_date = df_cwn['news_date'].to_list()
# df_cwn_text = df_cwn['news_text_sm'].to_list()
# df_cwn_date_n_text = {}
# for date, text in zip(df_cwn_date, df_cwn_text):
#     df_cwn_date_n_text[date] = text
                          
# print('dict zip : ', df_cwn_date_n_text)

# df_cwn_text

In [30]:
df_bloter_dnt

,news_date,title_n_text
0,2023-05-30 17:10,"웹3 메타버스 캐리버스 시네마틱 공개와 동시에 사전예약 돌입,㈜캐리버스가 웹3 메타..."
1,2023-05-30 13:15,"한국레노버 8세대 요가 신제품 3종 출시,레노버 요가 프로 9i한국레노버(대표 신규..."
2,2023-05-30 13:14,"SK텔레콤 양자보안 폰 ‘갤럭시 퀀텀4’ 출시,SK텔레콤(대표이사 사장 유영상)은 ..."
3,2023-05-30 12:07,"Arm 차세대 GPU·CPU 공개…모바일 기기 성능 업그레이드 기대,출처: Arm로..."
4,2023-05-30 10:12,"아이폰16 프로 모델 2종 디스플레 크기 커질까?,출처: Apple맥루머스 GSM ..."
...,...,...
1650,2022-05-31 16:29,"싱가포르 기업 84% ""코딩 실력 있는 지원자 선호해"",마카오 비즈니스닷컴이 싱가포..."
1651,2022-05-31 16:16,"IBM 맥도날드 AI 드라이브 스루 챗봇에 자연어 처리 기반 주문 기능 추가,영국 ..."
1652,2022-05-31 14:14,"印 델리 AI 기반 교통관리 시스템 배포 예고,인도 테크 매체 애널리틱스 인디아 매..."
1653,2022-05-31 14:01,"프라다 명품 브랜드의 디지털 자산 발행 행렬 동참…타임캡슐 NFT 런칭 예고,출처:..."


In [31]:
# f = open("../know_arc_pjt/data_crawling_D/data_sql_total_edited/total_news_t_202306221233.csv", "r", encoding='utf8')
# reader = csv.reader(f)

# for row in reader:
#     data = list(reader)
# # print(data)
# f.close()
# # dataframe으로 만들기
# df_total_t = pd.DataFrame(data=data, columns=['news_date', 'news_title', 'news_text_sm', 'url_in', 'news_writer', 'tags_string', 'thumb_addr', 'news_site'])  
# df_total_t

In [32]:
# 날짜당 제목에서 추출한 키워드 목록
def get_date_text(fname):
    # 튜토리얼에서 이용하는 `fname` 파일은 영화평과 평점이 \t 으로 구분된 two column tsv 파일입니다.
    # 예시는 이 cell 의 output 을 참고하세요.
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f] 
        docs = [doc for doc in docs if len(doc) == 2]

        if not docs:
            return [], []
        
        dates, texts = zip(*docs)
        return dates, texts

# cwn.kr data (df_cwn_date_n_text)
fname = '../data_analysis_test/test_data/cwn_dnt.txt'
# 이걸로 
dates, texts = get_date_text(fname)
# dates = list(dates)
# texts = list(texts)
# 위쪽의 5개 출력하기
# with open(fname, encoding="utf-8") as f:
#     for _ in range(5):
#         print(next(f).strip())

In [33]:
# dates
# texts

In [34]:
# KR-WordRank에 필요한 라이브러리
import sys
import re
sys.path.append('../')
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
import krwordrank
# print(krwordrank.__version__)

단어 추출에 영어/숫자를 포함할 예정이라면 normalize함수를 이용하여 텍스트를 normalize할 것

In [35]:
with open('../data_analysis_test/test_data/cwn_dnt_norm.txt', 'w', encoding='utf-8') as f:
    for date, text in zip(dates, texts):
        text = normalize(text, english=True, number=True)
        # 특수문자는 제거
        text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", text)
        f.write('%s\t%s\n' % (date, text))

## 해당 데이터에서 핵심 키워드들을 뽑아보자

In [36]:
# df_cwn_date_n_text
fname = '../data_analysis_test/test_data/cwn_dnt_norm.txt'
dates_n, texts_n = get_date_text(fname)
# 여기까진 dates_n, texts_n이 서로 분리되어있음

In [37]:
wordrank_extractor = KRWordRank(
    min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

# 여기서 for문
# text_key_list = []
# for text in texts_n:
keywords, rank, graph = wordrank_extractor.extract(texts_n, beta, max_iter)
    

scan vocabs ... 
num vocabs = 39297
done = 10


In [37]:
# keywords

위와 같이 vocabulary를 미리 설정하거나 decaying factor를 단어별로 다르게 (bias) 할당할 수 있으며, 모든 단어의 랭킹의 총 합은 vocabulary size와 같음. 즉 default decaying factor는 1.0

In [38]:
# 불용어 파일 읽기
stop_words = []
with open('../data_analysis_test/불용.txt', 'r', encoding='utf-8') as file:
   stop_words = file.read().splitlines()
# keyword_list
# # 불용어 제거
pop_list = []
for keyword in keywords:
    # print(keyword)
    # print(type(keyword))
    if keyword in stop_words:
        pop_list.append(keyword)
        # print(keyword)
        
# print(pop_list)
for p in pop_list:
    # print(p)
    keywords.pop(p, None)

# # filtered_tokens =  [token for token in keyword_list[0] if token not in stop_words]
# # filtered_tokens
# keywords

In [40]:
top_keywords = []
# for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
#     print('%8s:\t%.4f' % (word, r))
top_keywords.append(
    sorted(keywords.items(),
           key=lambda x:x[1],
           reverse=True)[:100]
)


keyword_list = []
for keywords in top_keywords:
    words, ranks = zip(*keywords)
    print(words)
    
    keyword_list.append(words)


('언어', '개발', '프로', 'ai', '데이터', '기업', '코딩', '한편', '보안', '순위', '구글', '매체', '코드', '플랫폼', 'tiobe', '애플', '출처', '서비스', 'nft', '지수', '지원', '인기', '기능', '분석', '테크', '티오베', '디지털', '시장', '학습', '메타', '문제', '전문', 'pyp', '교육', '검색', '생성', '파이썬', '공식', '중국', '이미', '게임', '결과를', '연구', '이에', '정보', 'language', '자바', '투자', '이어', '사이버', '기존', '가능', '신규', '시스템', '제품', '출시', '과정', '말했다', '소프트', '암호화', 'in', '가상', 'programming', '사실을', '스마트', '기준', '일부', '인공', '증가', '모델', '것을', '운영', '추가', '업계', '공격', '공개', '매우', '컴퓨터', '오픈', '도움을', '작업', '기기', '온라인', '그는', '로봇', '글로벌', '고객', '자동', '마이', '콘텐츠', '필요', '달러', 'and', '실제', 'ceo', '개인', '영상', '분야', 'it', 'ar')


In [ ]:
for k in range(100):
    
    message = '  --  '.join(
        ['%8s (%.3f)' % (top_keywords[0][k][0],top_keywords[0][k][1])]
         )
    print(message)
# 여기까지가 기사 본문으로부터 키워드 추출

 top 100에서 중복되는 키워드들을 제거하고 차이가 있는 키워드만 추출해서 살펴보겠습니다.

In [42]:
keyword_counter = {}

w_l = []
r_l = []
for keywords in top_keywords:
    words, ranks = zip(*keywords)
    for word in words:
        keyword_counter[word] = keyword_counter.get(word, 0) + 1
        w_l.append(word)
    for rank in ranks:
        r_l.append(rank)

csv_list = list(zip(w_l, r_l))
common_keywords = {word for word, count in keyword_counter.items() if count == 3}
len(common_keywords)

0

In [70]:
# csv_list


In [43]:
df_csv = pd.DataFrame(data=csv_list, columns=['words','ranks'])
df_csv = df_csv.sort_values(by=['ranks'], ascending=False)
df_csv

,words,ranks
0,언어,76.776393
1,개발,60.597695
2,프로,56.461848
3,ai,48.242664
4,데이터,44.364243
...,...,...
95,개인,8.788578
96,영상,8.783965
97,분야,8.690434
98,it,8.562951


In [44]:
df_csv.to_csv("../data_crawling_D/keywords/cwn_keywords.csv", mode='w')

## 키워드 분석

## # Reference keywords

In [48]:

# df_techworld
df_techworld_tnt = df_techworld[['news_date', 'total_keywords']]
# df_techworld_tnt

In [ ]:
# 데이터프레임에서 '내용'과 '작성일' 열을 가져옴
import re
from collections import Counter

# 모듈화하기
def get_keyword_monthly_agg(df_techworld_tnt):
    text_data = df_techworld_tnt['total_keywords']
    date_data = df_techworld_tnt['news_date']
    
    
    # 단어 등장 횟수를 저장할 딕셔너리 초기화
    word_counts = {}
    # 월별 단어 집계
    word_n_cnt_list = []
    
    # 데이터프레임을 순회하며 월 별로 단어 등장 횟수 집계
    for text, date in zip(text_data, date_data):
        # 텍스트 토큰화 및 형태소 분석
        
        # 년 추출
        tokens = text.split(",")
        year = date.split('-')[0]
        # 월 추출
        month = date.split('-')[1]
        year_month = year + '-' + month

        # 단어 등장 횟수 집계
        if year_month in word_counts:
            for token in tokens:
                word_counts[year_month][token] = word_counts[year_month].get(token, 0) + 1
        else:
            word_counts[year_month] = {}
            for token in tokens:
                word_counts[year_month][token] = 1
                
    # 횟수가 2이상인 단어만 추출하기
    min_word_count = 2
    top_words_limit = 21 # 최대 20개
    
    # 월 별로 상위 단어 추출
    
    for year_month, counts in word_counts.items():
        # print(f"{month}월")
        # 횟수만 추출하기
        sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        
        # 횟수가 2이상인 단어 필터링
        filtered_counts = [(word, count) for word, count in sorted_counts if count >= min_word_count]
        
        # 만약 2이상인 단어의 수가 5개 미만이면 모든 단어 추출
        if len(filtered_counts) < top_words_limit:
            filtered_counts = sorted_counts
        # 상위 단어 출력
        top_words = filtered_counts[:top_words_limit]
        for word, count in top_words:
            # print(f" {word} | 등장횟수: {count}")
            word_n_cnt_list.append([year_month, word, count])
        # print(word_n_cnt_list)
        print()
    
    return word_n_cnt_list

tw_key_list = get_keyword_monthly_agg(df_techworld_tnt)
# df_tw_key_h_list

In [55]:
# 리스트를 df으로
df_tw_key_list = pd.DataFrame(tw_key_list, columns=['date','words','cnt'])
print('2차원 리스트로 만든 dataframe:\n', df_tw_key_list)

2차원 리스트로 만든 dataframe:
         date words  cnt
0    2023-05  클라우드   34
1    2023-05    출시   20
2    2023-05    기반   18
3    2023-05   글로벌   18
4    2023-05   솔루션   16
..       ...   ...  ...
247  2022-06    테크   26
248  2022-06   데이터   26
249  2022-06   파트너   24
250  2022-06    지원   22
251  2022-06    지능   22

[252 rows x 3 columns]


In [ ]:
# pivot으로 {인덱스 : date, 컬럼: words, 값: cnt}인 테이블 생성
df_tw_key_pivot = df_tw_key_list.pivot(index='date', columns='words', values='cnt')
df_tw_key_pivot.fillna('0', inplace=True) # 결측값 0처리
df_tw_key_pivot = df_tw_key_pivot.apply(pd.to_numeric)
# df_tw_key_pivot

In [ ]:
df_tw_key_pivot.info()

In [68]:
# 각 단어당 cnt 합산을 구한 뒤 상위 100개 리스트에 담기
df_tw_key_pivot_sum = df_tw_key_pivot.sum()
df_tw_key_pivot_sum_srt = df_tw_key_pivot_sum.sort_values(ascending=False)
df_tw_key_pivot_sum_srt

words
클라우드      452.0
출시        362.0
솔루션       349.0
서비스       300.0
사업        263.0
          ...  
공략          6.0
공공          6.0
이스트소프트      6.0
유공          6.0
티맥스소프트      6.0
Length: 65, dtype: float64

In [69]:
# 인덱스에서 word를 빼내고
df_tw_key_pivot_sum_srt = df_tw_key_pivot_sum_srt.reset_index(drop=False)

In [73]:
ref_key_list = df_tw_key_pivot_sum_srt['words'].to_list()
# ref_list

In [74]:
# 100개 reference용 키워드 추출
top_100_ref_list = ref_key_list[:100]    

In [72]:
top_100_kw_list = []
for k in range(100):
    # print(top_keywords[0][k][0])
    top_100_kw_list.append(top_keywords[0][k][0])
print(top_100_kw_list)

['ai', '클라우드', '기업', '서비스', '솔루션', '데이터', '플랫폼', '디지털', '보안', '개발', '글로벌', '출시', '사업', '국내', '전문', '체결', '시장', '지원', '선정', '프로', '구축', '고객', '관리', '시스템', '업무', '발표', '운영', '한국', 'it', '제품', '산업', '메타', '대표', '서울', '비즈니스', '인공', '진행', '분석', '개최', '소프트', '인증', '파트너', '공급', '스마트', '분야', '연구', '로봇', '혁신', '통합', '스타트업', '협력', '자사', '기능', '투자', '올해', '공개', '교육', '환경', '자동', '획득', '인프라', '정보', '기존', '공공', '콘텐츠', '확대', '강화', '온라인', '오픈', '금융', '코리아', '구글', '확장', '영상', '모델', '3d', '추진', 'kt', '참가', 'mo', 'nft', '차세대', '6월', '성장', '가상', '신규', '인텔', '적용', '오라클', '마이', '가능', '가트너', '버추얼', '애플리케이션', 'digital', '의료', '공식', '매출', 'software', 'aws']


## ROUGE-1,2,l 로 성능 평가

In [76]:
from rouge import Rouge

# 모델로 구한 키워드들
rank_list = top_100_kw_list
rank_str = " ".join(rank_list)
# 실제 헤드라인과 내용에서 집계한 키워드들
ref_str = " ".join(top_100_ref_list)
model_out = [rank_str]
# '다양한', '관련', '것이', '새로운', '최근', '분석', '출처:', '활용', '따라', '한다.', '세계', '공동', '가장', '하는', '현재', '기존', '등을', '점유율은', '모든', '다른', '대해', '많은', '순위', '결과를', '또한', '대비', '10', '신규', '이용', '이를', '주요', '모두', '문제', '그러나', '여러', '출시', '이미', '생성', '가능', '이어', '공개', '발표', '이후', '최신', '전했다', '특히', '이상', '인기', '이는', '하지만', '지수', '이러한', '때문에', '핵심', '6월', '아이', '것은', '그리고', '이에', '같이', '동시에', '각각', '참여', '시간', '전체', '하나', '수준', '물론', '다수', '동안', '9월', '어떤', '전망', '혹은', '양자', '전자', '바로', '여전히'
reference = [ref_str]
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

# >>>
# {   'rouge-1': {   'f': 0.6279006234427593,
#                    'p': 0.8604497354497355,
#                    'r': 0.5273531655225019},
#     'rouge-2': {   'f': 0.3883256484545606,
#                    'p': 0.5244559362206421,
#                    'r': 0.32954545454545453},
#     'rouge-l': {   'f': 0.6282785202429159,
#                    'p': 0.8122895622895623,
#                    'r': 0.5369305616983636}}

{'rouge-1': {'r': 0.6307692307692307, 'p': 0.41, 'f': 0.4969696921946741},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.2153846153846154, 'p': 0.14, 'f': 0.16969696492194689}}